In [17]:
from pathlib import Path
from pydicom import dcmread
import glob
import pandas as pd

# Which data attributes to read from the dicom files.
DATA_ATTRIBUTES = ['SeriesInstanceUID', 'StudyDate', 'StudyTime', 'Modality', 'PatientID', 'PatientBirthDate', 'PatientSex', 'PatientAge', 'PatientSize', 'PatientWeight']

def folder_dcm_filenames(dir_path, recursive=True):
    """
    Creates generator with Path objects for each DICOM file in the specified directory.

    Args:
        dir_path (str, Path): Path to DICOM data directory.
        recursive (bool): If True, look through directory recursively.

    Returns:
        Generator with DICOM file Path objects.
    """
    assert type(dir_path) in [str, Path], 'Path to directory must be string or Path object'
    dir_path = Path(dir_path)
    if recursive:
        dcm_files = dir_path.rglob('*.dcm')
    else:
        dcm_files = dir_path.glob('*.dcm')
    return dcm_files

def read_dcm_folder(dir_path, recursive=True):
    """
    Creates a generator with parsed DICOM files.

    Args:
        dir_path (str, Path): Path to DICOM data directory.
        recursive (bool): If True, look through directory recursively.

    Yields:
        List of pydicom.dataset.FileDataset objects.
    """
    assert type(dir_path) in [str, Path], 'Path to directory must be string or Path object'
    dcm_files = folder_dcm_filenames(dir_path, recursive=recursive)
    for fpath in dcm_files:
        yield dcmread(fpath), fpath

def dcm_ecg_annotations(dcm):
    """
    Read annotations in DICOM, ignoring Pacemaker Spike annotations.

    Args:
        dcm (pydicom.dataset.FileDataset): DICOM file.

    Returns:
        Dictionary: Dictionary with (annotation_name, annotation_value) key-value pairs.
    """
    # Check if annotations are present
    try:
        annot_sequence = dcm['WaveformAnnotationSequence']
    except:
        return None
    annotations = dict()
    # For each annotation, try if there are basic automatic annotations (e.g. heart rate, PR Interval, T Axis)
    for annot in annot_sequence:
        try:
            name = annot.ConceptNameCodeSequence[0].CodeMeaning
            if name == 'Pacemaker Spike': continue # Ignore pacemaker annotations for now
            value = float(annot.get('NumericValue'))
        except:
            continue
        annotations[name] = value
    return annotations

def dcm_to_dict(dcm):
    """
    Return DICOM attributes as a dictionary.

    Args:
        dcm (pydicom.dataset.FileDataset): DICOM file.

    Returns:
        Dictionary: Dictionary with (attribute_name, attribute_value) key-value pairs.
    """
    data = {attr: dcm.get(attr, default=None) for attr in DATA_ATTRIBUTES}
    data['NumECGs'] = 1
    waveform = dcm.WaveformSequence[0]
    data['SampleDuration'] = waveform.NumberOfWaveformSamples
    data['SamplingFrequency'] = waveform.SamplingFrequency
    data['NumChannels'] = waveform.NumberOfWaveformChannels
    # Update data dictionary with ecg annotations
    annotations = dcm_ecg_annotations(dcm)
    if annotations is not None: data = {**data, **annotations}
    return data

def read_dcm_file(path):
    """Read/parse DICOM file using pydicom libary."""
    dcm = dcmread(path)
    return dcm

def df_from_dcm(dcm_generator, use='all'):
    """
    Create pandas DataFrame from DICOM files.

    Args:
        dcm_generator (generator): Generator for (pydicom.dataset.FileDataset, file_path) tuples.
        use (str): One of ['all', 'first', 'latest'], indicates which dicom files to include
            per patient, all, only first, or chronologically last study. Default to 'all'.

    Returns:
        pandas.Dataframe object containing dicom file information.
    """
    assert use in ['all', 'first', 'latest'], 'Argument \'use\' must be one of [all, first, latest].'

    datadict = dict() # Dictionary seriesID or patientID : dictionary with dcm data
    patient2date = dict() # Dictionary patientID : (StudyDate, list(seriesIDs))

    # Enumerate over DICOM files
    for i, (dcm, path) in enumerate(dcm_generator):
        # Acquire data
        seriesID = dcm.SeriesInstanceUID
        patientID = dcm.PatientID
        date = dcm.StudyDate
        data = dcm_to_dict(dcm)
        data['FileName'] = path

        # Use all patient ECGs, dictionary becomes seriesID oriented
        if use == 'all':
            assert seriesID not in datadict, f'Duplicate seriesID {seriesID}'
            datadict[seriesID] = data
        # For each patient, use only the first or latest ECG, dictionary patientID oriented
        else:
            # If patientID not seen before, use this ECG
            if patientID not in patient2date:
                patient2date[patientID] = (date, [seriesID])
                datadict[patientID] = data
            # If patientID already seen before, use either latest or first ECG
            else:
                prev_date, seriesIDs = patient2date[patientID]
                assert seriesID not in seriesIDs, f'Duplicate seriesID {seriesID}'
                seriesIDs.append(seriesID)
                if (use == 'first' and date < prev_date) or (use == 'latest' and date > prev_date):
                    patient2date[patientID] = (date, seriesIDs)
                    datadict[patientID] = data
                else:
                    patient2date[patientID] = (prev_date, seriesIDs)
                datadict[patientID]['NumECGs'] = len(seriesIDs)
    # Create Dataframe from the dictionaries
    df = pd.DataFrame.from_dict(datadict, orient='index')
    # Reset index to integers, patientID and seriesID already in dataframe
    df.reset_index(drop=True, inplace=True)
    return df



data_path = '\ECG'
dcm_gen = read_dcm_folder(data_path, recursive=True)

df = df_from_dcm(dcm_gen, use='all')
print(len(df))
print(df.head())

#Save the log
df.to_excel("ECG_log.xlsx")  


106356
                           SeriesInstanceUID StudyDate StudyTime Modality  \
0  9.198534941724483858745970131990876567594  20040206    092547      ECG   
1  9.192582106649194121789830882166586114101  20040206    092621      ECG   
2  9.233224299430102423345417558810087641914  20040408    135907      ECG   
3  9.234798769551807449878750887115357671682  20120725    115258      ECG   
4  9.257768287114138921150197710417829875091  20130731    111147      ECG   

   PatientID PatientBirthDate PatientSex PatientAge  PatientSize  \
0  DIST_2881         19300101       None       073J          NaN   
1  DIST_2881         19300101       None       073J          NaN   
2  DIST_2881         19300101       None       074J          NaN   
3  DIST_2881         19300101       None       082J         1.61   
4  DIST_2881         19300101       None       083J          NaN   

   PatientWeight  ...  QRS Duration  QT Interval  RR Interval  QTc Interval  \
0            NaN  ...         132.0       

In [51]:
print('Follow-up of 1 year\n', df['SamplingFrequency'].value_counts())

Follow-up of 1 year
 250.0    60923
500.0    45433
Name: SamplingFrequency, dtype: int64


In [66]:
import h5py
import numpy as np
from tqdm import tqdm
import os

samples=5000
channels=12

hz500set = df.loc[df['SamplingFrequency'] == 500.0]
hz250set = df.loc[df['SamplingFrequency'] == 250.0]

# Fixed ordering of leads
LEAD_ORDER = ['Lead I (Einthoven)', 'Lead II', 'Lead III', 'Lead aVR', 'Lead aVL', 'Lead aVF', 'Lead V1', 'Lead V2', 'Lead V3', 'Lead V4', 'Lead V5', 'Lead V6']

path = '\ECG'

def read_dcm_file(path):
    """Read/parse DICOM file using pydicom libary."""
    dcm = dcmread(path)
    return dcm

def create_hdf5(outpath, group_name, filenames, samples=2500, channels=12):
    """
    Creates/extends a HDF5 file and fills the corresponding group with ECGs
        from DICOM files.
    Args:
        outpath (str): Path for the HDF5 file.
        group_name (str): Group or dataset name in the HDF5 file to save ECGs to.
        filenames (list): List of paths to the DICOM files.
        samples (int): Number of recorded time points to read/store.
        channels (int): Number of channels\leads for the ECGs.
    """
    N = len(filenames)
    with h5py.File(outpath, 'a') as f:
        # Store ecg data
        dset_data = f.create_dataset(group_name, (N, samples, channels), dtype='f4')
        for i, fpath in tqdm(enumerate(filenames), total=len(filenames)):
            dcm = read_dcm_file(fpath)
            ecg = dcm.waveform_array(0)
            # Map each lead to an index in the array (to ensure uniform lead-order across all ecgs)
            lead2idx = {}
            for j, channel in enumerate(dcm.WaveformSequence[0].ChannelDefinitionSequence):
                source = channel.ChannelSourceSequence[0].CodeMeaning
                lead2idx[source] = j
            try:
                indices = [lead2idx[lead] for lead in LEAD_ORDER] # Try for "Lead I (Einthoven)", "Lead II" notation
            except KeyError:
                indices = [lead2idx[lead.split()[1]] for lead in LEAD_ORDER] # Try for "I", "II" notation
            dset_data[i, :, :] = ecg[:samples, indices]

outpath= '250hz_unpreprocessed.h5' 
filenames=list(hz250set['FileName'])

create_hdf5(outpath, 'hdf5', filenames, samples=2500, channels=12)

# hz500set.to_excel("ECG_log_500hz.xlsx")  
# hz250set.to_excel("ECG_log_250hz.xlsx")  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60923/60923 [48:42<00:00, 20.85it/s]


In [70]:
data_path = '500hz_unpreprocessed.h5'
f = h5py.File(data_path, 'r')


In [74]:
import h5py

data_path = '250hz_unpreprocessed.h5'
data = h5py.File(data_path, 'r')['hdf5']

print(data)

import math
import numpy as np
from tqdm import tqdm

def preprocess_data(data_x, smooth_window_size=51, smooth_order=4, fourier_baseline_resolution=20, verbosity=False):
    """ function: preprocess_data
    preprocess the data by smoothing and straightening.
    Args:
        data_x : np.ndarray
            the data to preprocess.
    Returns:
        p_data_x : np.ndarray
            preprocessed data
    """
    assert data_x.ndim == 3

    if verbosity:
        print("Preprocessing data...")
        start = time.time()

    p_data_x = np.empty(shape=data_x.shape)

    for i, ecg in tqdm(enumerate(data_x)):
        for channel in range(ecg.shape[1]):
            prepped_channel = savitzky_golay(
                ecg[:, channel],
                window_size = smooth_window_size,
                order = smooth_order
            )
            prepped_channel = fourier_straighten(
                prepped_channel,
                resolution = fourier_baseline_resolution
            )
            p_data_x[i, :, channel] = prepped_channel
        
            
    if verbosity:
        print('\nDone, took ' + str(round(time.time() - start, 1)) + ' seconds')
    return p_data_x

def savitzky_golay(y, window_size=51, order=4, deriv=0, rate=1):
    r"""Smooth (and optionally differentiate) data with a Savitzky-Golay filter.
    The Savitzky-Golay filter removes high frequency noise from data.
    It has the advantage of preserving the original shape and
    features of the signal better than other types of filtering
    approaches, such as moving averages techniques.
    Parameters
    ----------
    y : array_like, shape (N,)
        the values of the time history of the signal.
    window_size : int
        the length of the window. Must be an odd integer number.
    order : int
        the order of the polynomial used in the filtering.
        Must be less then `window_size` - 1.
    deriv: int
        the order of derivative to compute (default = 0 means only smoothing)
    Returns
    -------
    ys : ndarray, shape (N)
        the smoothed signal (or it's n-th derivative).
    Notes
    -----
    The Savitzky-Golay is a type of low-pass filter, particularly
    suited for smoothing noisy data. The main idea behind this
    approach is to make for each point a least-square fit with a
    polynomial of high order over a odd-sized window centered at
    the point.
    Examples
    --------
    t = np.linspace(-4, 4, 500)
    y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
    ysg = savitzky_golay(y, window_size=31, order=4)
    import matplotlib.pyplot as plt
    plt.plot(t, y, label='Noisy signal')
    plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
    plt.plot(t, ysg, 'r', label='Filtered signal')
    plt.legend()
    plt.show()
    References
    ----------
    .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
       Data by Simplified Least Squares Procedures. Analytical
       Chemistry, 1964, 36 (8), pp 1627-1639.
    .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
       W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
       Cambridge University Press ISBN-13: 9780521880688
    Note: this code was copied from:
    https://scipy.github.io/old-wiki/pages/Cookbook/SavitzkyGolay
    """
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * math.factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

def fourier_straighten(signal, resolution=20):
    """ function: fourier_straighten
    straighten an ecg using a low resolution to obtain ecg baseline, and then
    straightening the ecg by setting the baseline to zero
    Args:
        signal : np.ndarray
            the ecg signal to straighten
        resolution : int [optional, default: 20]
            the resolution of the baseline. if set to high important details
            from the ecg will be removed, recommended to keep under 30
    Returns:
        corrected_ecg : np.ndarray
            the straightened array
    """
    assert signal.ndim == 1
    c = get_fourier_coefficients(signal, resolution)
    baseline = reconstruct_wave(coefficients=c, signal_length=signal.shape[0])

    corrected_ecg = np.subtract(signal, baseline)
    return corrected_ecg

def get_fourier_coefficients(signal, resolution=600):
    """ function: get_fourier_coefficients
    perform fourier series analysis on a signal and return the nth coefficient
    for a function that approximate the wave
    assumes the data is modelled by an even function (f(x) = f(-x)).
    the function on this page is followed (the second one under cosine series)
    https://en.wikipedia.org/wiki/Fourier_sine_and_cosine_series.
    the function that reconstructs this wave is: reconstruct_wave
    Args:
        signal : np.ndarray
            a 1d array containing the signal to perform the analysis on
        resolution : int [optional, default: 600]
            the amount of coefficients to return. the higher the resolution the
            higher the fidelity of the reconstructed wave
    Returns:
        coefficients : np.ndarray
            a 1d array containing the coefficients that model the given data
    """
    # only works if array is 1d
    assert signal.ndim == 1
    linspace = np.linspace(0, 1, len(signal)).reshape((-1, 1))
    pi_n = np.pi * np.arange(resolution).reshape((1, -1))
    tmp1 = np.cos(linspace @ pi_n)
    tmp2 = signal.reshape((-1,1)) * tmp1
    coefficients = (2/len(signal)) * np.sum(tmp2, axis=0)
    return coefficients

def reconstruct_wave(coefficients, signal_length):
    """ function: reconstruct wave
    reconstructs a wave using a given set of coefficients using the function as
    seen on this page (first equation under cosine series)
    https://en.wikipedia.org/wiki/Fourier_sine_and_cosine_series
    Args:
        coefficients : np.ndarray
            a 1d array of coefficients to reconstruct wave from, such as those
            given by the function get_fourier_coefficients
        signal_length : int
            the target length of the reconstructed array
    Returns:
        reconstruction : np.ndarray
            a 1d array of length signal_length that approximates the data of
            which the coefficients were extracted
    """
    x = np.arange(signal_length).reshape((-1,1))
    n = np.arange(len(coefficients)).reshape((1,-1))
    tmp1 = np.cos(((x @ n) * np.pi) / signal_length)
    coefficients[0]  = coefficients[0] / 2
    tmp2 = tmp1 * coefficients
    reconstruction = np.sum(tmp2, axis=1)
    return reconstruction


new_path = '250hz_preprocessed.hdf5'
with h5py.File(new_path, 'a') as f:
    # Store ecg data
    dset_data = f.create_dataset('data', data.shape, dtype='f4')
    batch_size = 100
    for i in range(0, len(data), batch_size):
        print(f'{i}/{len(data)}')
        p_data = preprocess_data(data[i:i+batch_size])
        dset_data[i:i+batch_size] = p_data

<HDF5 dataset "hdf5": shape (60923, 2500, 12), type "<f4">
0/60923


0it [00:00, ?it/s]C:\Users\mzkolk\AppData\Local\Temp/ipykernel_17676/2367740184.py:99: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  window_size = np.abs(np.int(window_size))
C:\Users\mzkolk\AppData\Local\Temp/ipykernel_17676/2367740184.py:100: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curren

100/60923


100it [00:02, 38.32it/s]


200/60923


...

100it [00:02, 38.13it/s]


60800/60923


100it [00:02, 40.50it/s]


60900/60923


23it [00:00, 40.28it/s]
